In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import asyncio
import json

from loguru import logger


loop = asyncio.get_running_loop()

In [3]:
with open('settings/mainnet.json', 'r') as f:
    tonlib_config = json.load(f)
    
tonlib_config['liteservers'] = tonlib_config['liteservers']
print('Num ls:', len(tonlib_config['liteservers']))

Num ls: 20


In [4]:
os.makedirs('private/ton_keystore', exist_ok=True)

# TonlibMulticlient

In [5]:
loop = asyncio.get_running_loop()

In [6]:
from tApi.tonlib.multiclient import TonlibMultiClient



liteserver_blacklist = [3, 6, 13]
multiclient = TonlibMultiClient(loop, 
                                tonlib_config, 
                                './private/ton_keystore',
                                liteserver_blacklist=liteserver_blacklist)
await multiclient.init(0)

2022-03-15 23:58:31.414 | WARNING  | tApi.tonlib.multiclient:init:43 - Skipping blacklisted liteserver #003
2022-03-15 23:58:31.416 | WARNING  | tApi.tonlib.multiclient:init:43 - Skipping blacklisted liteserver #006
2022-03-15 23:58:31.421 | WARNING  | tApi.tonlib.multiclient:init:43 - Skipping blacklisted liteserver #013
[ 4][t 1][2022-03-15 23:58:31.430646700][TonlibClient.cpp:1479][!Tonlib][&tonlib_query]	Tonlib got query [id:1] setLogVerbosityLevel {
  new_verbosity_level = 0
}
[ 4][t 1][2022-03-15 23:58:31.430707000][TonlibClient.cpp:1518][!Tonlib][&tonlib_query]	Tonlib got static query setLogVerbosityLevel {
  new_verbosity_level = 0
}
2022-03-15 23:58:31.475 | INFO     | tApi.tonlib.client:init:95 - TonLib #002 inited successfully
2022-03-15 23:58:31.480 | INFO     | tApi.tonlib.client:init:95 - TonLib #001 inited successfully
2022-03-15 23:58:31.503 | INFO     | tApi.tonlib.client:init:95 - TonLib #008 inited successfully
2022-03-15 23:58:31.512 | INFO     | tApi.tonlib.client:

In [7]:
await multiclient.getMasterchainInfo()

{'@type': 'blocks.masterchainInfo',
 'last': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 19073995,
  'root_hash': 'rIjOInBdfaCzFQPigpD1F6KfVnJMA1VO31vM+QHnc8s=',
  'file_hash': 'nIflOIlztjRmmpAECLh7WoHBmeO4K61zlwzaEhDD8ns='},
 'state_root_hash': 'aDM1iMZLjZtxxVxE6MnypRA/lXXgucg3zlHGMc7/VwA=',
 'init': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '0',
  'seqno': 0,
  'root_hash': 'F6OpKZKqvqeFp6CQmFomXNMfMj2EnaUSOXN+Mh+wVWk=',
  'file_hash': 'XplPz01CXAps5qeSWUtxcyBfdAo5zVb1N979KLSKD24='},
 '@extra': '1647388723.2816806:9:0.48574086975954733'}

In [8]:
from ipywidgets import Output

output = Output()
async def monitor():
    while True:
        res = ''
        for client in multiclient.clients.values():
            res += f'{client.ls_index:2d}: {len(client.tonlib_wrapper.futures):3d}\n'
        with output:
            output.clear_output(wait=True)
            print(res)
        await asyncio.sleep(0.25)
        
display(output)
asyncio.ensure_future(monitor())

Output()

<Task pending name='Task-56' coro=<monitor() running at /tmp/ipykernel_60293/22469496.py:4>>

In [10]:
import time
from tqdm.auto import tqdm


for i in tqdm(range(16384)):
    fut = multiclient.getBlockTransactions(-1, -9223372036854775808, 2000, 10)
    asyncio.ensure_future(fut)
    await asyncio.sleep(0.0005)

  0%|          | 0/16384 [00:00<?, ?it/s]

2022-03-15 23:59:39.909 | INFO     | tApi.tonlib.client:reconnect:47 - Client #{self.ls_index:03d} has no reconnect attempts left
Task exception was never retrieved
future: <Task finished name='Task-16480' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrie

Task exception was never retrieved
future: <Task finished name='Task-16601' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-16634' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

Task exception was never retrieved
future: <Task finished name='Task-16794' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-16795' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

Task exception was never retrieved
future: <Task finished name='Task-16847' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-16849' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

...

Task exception was never retrieved
future: <Task finished name='Task-17352' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-17355' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

Task exception was never retrieved
future: <Task finished name='Task-17384' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
2022-03-15 23:59:41.151 | INFO     | tApi.tonlib.client:reconnect:47 - Client #{self.ls_index:03d} has no reconnect attempts left
Task exception was never retrie

Task exception was never retrieved
future: <Task finished name='Task-17428' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-17432' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

Task exception was never retrieved
future: <Task finished name='Task-17451' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-17452' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

Task exception was never retrieved
future: <Task finished name='Task-17468' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/tonlib/multiclient.py:148> exception=AttributeError("'NoneType' object has no attribute 'execute'")>
Traceback (most recent call last):
  File "/workspace/tApi/tApi/tonlib/multiclient.py", line 150, in getBlockTransactions
    return await client.getBlockTransactions(workchain, shard, seqno, count, root_hash, file_hash, after_lt, after_hash)
  File "/workspace/tApi/tApi/tonlib/client.py", line 501, in getBlockTransactions
    fullblock = await self.lookupBlock(workchain, shard, seqno)
  File "/workspace/tApi/tApi/tonlib/client.py", line 477, in lookupBlock
    return await self.tonlib_wrapper.execute(request)
AttributeError: 'NoneType' object has no attribute 'execute'
Task exception was never retrieved
future: <Task finished name='Task-17469' coro=<TonlibMultiClient.getBlockTransactions() done, defined at /workspace/tApi/tApi/

In [ ]:
[3, 6, 13]

# TonlibClient

In [ ]:
loop = asyncio.get_running_loop()

In [ ]:
from tApi.tonlib.client import TonlibClient


client = TonlibClient(1, 
                      tonlib_config, 
                      './private/ton_keystore',
                      loop,
                      cdll_path=None,
                      verbosity_level=0)
await client.init()

In [ ]:
masterchain_info = await client.getMasterchainInfo()
last_block = masterchain_info['last'].copy()
last_block.pop('@type')

block_header = await client.getBlockHeader(**last_block)

transactions = await client.getBlockTransactions(**last_block, count=50)
transactions